# EDA Project 
Peter Goff

### Question/need: <br>
A number of analyses have shown that ridership on NYC's subway system (MTA) has decreased during the covid pandemic. While it is usefful to quantify the aggregate drop, it would also be helpful to identyify meaningful heterogeneity in ridership changes across the city. City, state, and federal health offices all stand to benefit from a more nuanced understanding of how ridership has changed as a result of social-distancing policy. For example, in areas where ridership has decreased dramatically, do we observe fewer covid outbreaks? Do we also observe more business forcloseures? <br>
Having information on the ways in which transit sites have deviated from historical traffic patterns in times of crisis may help governmental agencies direct resources (temporary hospitals, vaccines, etc.) and consider implementing alternative policies, such as limiting transactions at certain sites. Further, findings from such analyses could be used to shape these policies by providing viable thesholds and time targets. <br>
This project will examine heterogeneity in transit usage during covid by comparing transit numbers for July of 2020 and comparing them to historical usage from the three prior years with the goal of identifying the stations with greatest proportional decreases and increases. <br>

### Data Description: <br>
This project will draw from several publicly available data sets, centered primarily on [MTA Turnstyle data](http://web.mta.info/developers/turnstile.html). This data will be downloaded directly from the MTA website. Key characteristics of this data include the turnstyle location (station, linename, and division), timestamp, and entries/exits. While the data are recorded at the turnstyle-level, an aggregation to the station-level would likely provide a more meaninful portrayal of usage trends. <br>
At this initial stage of the project, I plan to provide a comprehensive graphical-descriptive portrayal of the heterogeneity in transit usage during covid. However, if time allows, subsequent analyses integrating business/commerce and/or demographic data may add fruitful nuance to the trends that emerge. <br>

### Tools: <br>
Data will be downloaded as local .csv files, then processed into a SQL database. That database will be querried into a Pandas dataframe for final analysis. Data cleaning and analysis will be conducted in Pandas and perhaps geopandas. Ideally, I would like to present my findings in their geographic context via a shapefile and appropriate graphing program. <br>

### MVP Goal: <br>
The minimum viable product will be a catapilliar plot that depicts proportional changes in usage across all MTA stations. Accompanying this plot will be documentation on the stations on the tails of the plot that experienced the greatest and smallest decreases (or increases, as the case may be). <br>


Codebook of [field descriptions](http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt): <br>
    
C/A      = Control Area (A002) <br>
UNIT     = Remote Unit for a station (R051) <br>
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00) <br>
STATION  = Represents the station name the device is located at <br>
LINENAME = Represents all train lines that can be boarded at this station <br>
           Normally lines are represented by one character.  LINENAME 456 NQR repersents train server for 4, 5, 6, N, Q, and R trains.  <br>
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND    <br>
DATE     = Represents the date (MM-DD-YY) <br>
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event <br>
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours) <br>
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities.  <br>
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered.  <br>
ENTRIES  = The comulative entry register value for a device <br>
EXIST    = The cumulative exit register value for a device <br>

It [appears](https://groups.google.com/g/mtadeveloperresources/c/CQI9eVdk4TU/m/A6aHplb2SxAJ) that UNIT = Booth and and C/A = Remote in the [Remote Unit/Control Area/Station Name Key](http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls) file. <br>

"Unfortunately, it’s not as simple as aggregating on the station column or the remoteunit column. There are situations where a single station has multiple names and multiple remotes. To get an accurate aggregation, we need to combine all of the turnstiles that are topologically connected by station platforms and corridors. The MTA website where the turnstile data is published comes with an excel spreadsheet called [Remote-Booth-Station](http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls) that should be able to help with this." ... "If we were to use this as a lookup table, we could associate the station with a turnstile, but would have no clear way to aggregate all of the records to a single station complex." ([source](https://medium.com/qri-io/taming-the-mtas-unruly-turnstile-data-c945f5f96ba0)) <br>
The author manually created a [crosswalk table](https://qri.cloud/nyc-transit-data/remote_complex_lookup) that allows “remote unit” identifiers found in the NY MTA Turnstile Data to be associated with station complex ids found in the MTA station list. 

# Create the database
```commandline
sqlite3 "////Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/mta_goff.db"
```

# Define transit tables
```sql
CREATE TABLE IF NOT EXISTS mta_data (
	CA TEXT,
	UNIT TEXT,
	SCP TEXT,
	STATION TEXT,
	LINENAME TEXT,
	DIVISION TEXT,
	DATE TEXT,
	TIME TEXT,
	DESC TEXT,
	ENTRIES INTEGER,
	EXITS INTEGER,
	PRIMARY KEY (CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS)
	);
```

# Define station table
```sql
CREATE TABLE IF NOT EXISTS mta_stations (
    "Station ID" INTEGER,
    "Complex ID" INTEGER,
    "GTFS Stop ID" TEXT,
    "Division" TEXT,
    "Line" TEXT,
    "Stop Name" TEXT,
    "Borough" TEXT,
    "Daytime Routes" TEXT,
    "Structure" TEXT,
    "GTFS Latitude" FLOAT,
    "GTFS Longitude" FLOAT,
    "North Direction Label" TEXT,
    "South Direction Label" TEXT,
    "ADA" INTEGER,
    "ADA Notes" TEXT,
	PRIMARY KEY ("Station ID", "Complex ID", "GTFS Stop ID", "Division", "Line")
	);
```

# Define body table
```sql
CREATE TABLE mta_body (
    remote TEXT,
    booth TEXT,
    complex_id INTEGER,
    station TEXT,
    line_name TEXT,
    divison TEXT,
	PRIMARY KEY (remote, booth, complex_id)
	);
```

In [51]:
import urllib.request

# These are the extensions for the July MTA data files
# The extensions are in the format [2 digit year][2 digit month][2 digit day]
url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
week_nums = [200704, 200711, 200718, 200725, 190727, 190720, 190713, 190706, 180728, 180721, 180714, 180707, 170729, 170722, 170715, 170708, 170701]

# Downloading turnstyle csv files from the MTA site to my EDA project folder within my git hub repo 
for week_num in week_nums:
    urllib.request.urlretrieve(f"http://web.mta.info/developers/data/nyct/turnstile/turnstile_{week_num}.txt",
                               f"/Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/turnstile_{week_num}.txt")

In [30]:
# Downloading MTA file that can be used to aggregate turnstyles 
urllib.request.urlretrieve('http://web.mta.info/developers/data/nyct/subway/Stations.csv',
                               f"/Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/Stations.csv");

Using terminal to append data files. The first line initializes the data using the first data set (excluding the first row of column names). The second line appends all other data to that file, working from the bottom up (-n) and stopping at the second line (+2), while supressing the output (-q).
```commandline
 head -1 turnstile_200704.txt > MTA_July_2017_20.csv
 tail -n +2 -q *.txt >> MTA_July_2017_20.csv
```

In [ ]:
# Go back to sqlite tab; load the big csv into sqlite
```sql
.mode csv
.import "/Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/MTA_July_2017_20.csv" mta_data 
.import "/Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/Stations.csv" mta_stations
.import "/Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/body.csv" mta_body
```

In [35]:
%pwd

'/Users/pgoff/Google Drive/1_Learning_Python/Metis DS'

In [36]:
pd.read_csv("/Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/MTA_July_2017_20.csv")

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,00:00:00,REGULAR,6233682,2110437
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,04:00:00,REGULAR,6233696,2110445
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,08:00:00,REGULAR,6233712,2110473
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,12:00:00,REGULAR,6233790,2110560
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,16:00:00,REGULAR,6233942,2110622
...,...,...,...,...,...,...,...,...,...,...,...
3434861,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,07/24/2020,05:00:00,REGULAR,5554,537
3434862,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,07/24/2020,09:00:00,REGULAR,5554,537
3434863,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,07/24/2020,13:00:00,REGULAR,5554,537
3434864,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,07/24/2020,17:00:00,REGULAR,5554,538


In [45]:
# Now to extract to pandas using sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("sqlite:////Users/pgoff/Google Drive/1_Learning_Python/Goff_Repo/EDA_Project/mta_goff.db")
mta_data = pd.read_sql('SELECT * FROM mta_data;', engine)

In [46]:
mta_stations = pd.read_sql('SELECT * FROM mta_stations;', engine)

In [48]:
mta_stations = mta_stations[1:] # removing redundant header line
mta_stations.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
1,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,,Manhattan,0,
2,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,
3,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,
4,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.76182,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,
5,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,


In [49]:
mta_body = pd.read_sql('SELECT * FROM mta_body;', engine)

,remote,booth,complex_id,station,line_name,divison
0,remote,booth,complex_id,station,line_name,division
1,R001,A060,635,WHITEHALL ST,R1,BMT
2,R001,A058,635,WHITEHALL ST,R1,BMT
3,R001,R101S,635,SOUTH FERRY,R1,IRT
4,R002,A077,628,FULTON ST,ACJZ2345,BMT


In [50]:
mta_body = mta_body[1:] # removing redundant header line
mta_body.head()

,remote,booth,complex_id,station,line_name,divison
1,R001,A060,635,WHITEHALL ST,R1,BMT
2,R001,A058,635,WHITEHALL ST,R1,BMT
3,R001,R101S,635,SOUTH FERRY,R1,IRT
4,R002,A077,628,FULTON ST,ACJZ2345,BMT
5,R002,A081,628,FULTON ST,ACJZ2345,BMT
